In [2]:
from ipynb.fs.full.common import *

Variables

In [3]:
# ['Hemlata', 'Malti', 'Preeti', 'Sharifa', 'Vinita', 'VKS'] ['Geeta', 'Jitendra', 'Jyoti', 'Kuldeep', 'Seema', 'VijayLaxmi']
ACTIVE_SHAM = 'Active'
SAMPLE = 'Hemlata'
ELECTRODES = '32electrodes' # '32electrodes' 
GROUP1 = 'Pre'
GROUP2 = 'Post' # generally GROUP1+1
mapping = {0: 'theta', 1: 'alpha', 2: 'beta'}

Importing all packages

In [4]:
import numpy as np
import pandas as pd
import mne
import matplotlib
import matplotlib.pyplot as plt
import os
from scipy.stats import ttest_rel
from mne_connectivity import spectral_connectivity_epochs
import networkx as nx
import numpy as np 
import glob

matplotlib.use('TkAgg')

## LOAD DATA

In [5]:
# # 2Pre-TDCS : .easy, # 3TDCS : .easy, # 4Post-TDCS : .easy
# import glob
# folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', ELECTRODES, ACTIVE_SHAM)
# # folders = ['1Pre-BML', '2Pre-TDCS', '3TDCS', '4Post-TDCS', '5Post-BML']
# # folders = ['2Pre-TDCS', '3TDCS', '4Post-TDCS']
# all_files = []
# # for folder in folders:
# #     ext = '.eeg' if folder[0] in ['1', '5'] else 'Close.easy'
# #     is_even = 1
# #     for _ in glob.glob(os.path.join(folder_path, folder, SAMPLE) + '/*' + ext):
# #         if (folder[0] == '3' and is_even%2==1) or folder[0] in ['2', '4']: #TODO: add for both 1 and 5 also 
# #             all_files.append(_)
# #         is_even += 1

# #     sorted_files = sorted(all_files, key=lambda x: os.path.getmtime(os.path.join(folder_path, folder, SAMPLE, x)))
# # print(len(sorted_files), sorted_files)
# folders = ['2Pre-TDCS', '4Post-TDCS']
# for folder in folders:
#     ext = 'Close.easy'
#     for _ in glob.glob(os.path.join(folder_path, folder, SAMPLE) + '/*' + ext):
#         all_files.append(_)

# print(len(all_files), all_files)

# # TODO: ADD iterator that changes Groups value and iterates (len(sorted_files) - 1 times)
# raw_1 = data_transformation_easy(all_files[0])
# raw_1 = g1_preprocess(raw_1)
# raw_2 = data_transformation_easy(all_files[1])
# raw_2 = g1_preprocess(raw_2)

In [6]:
folder_path = os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', ELECTRODES, ACTIVE_SHAM)
all_files = []

folders = ['2Pre-TDCS', '4Post-TDCS']
for folder in folders:
    ext = 'Close.easy' if ELECTRODES == '32electrodes' else 'EEG.easy'
    for _ in glob.glob(os.path.join(folder_path, folder, SAMPLE) + '/*' + ext):
        all_files.append(_)

# TODO: ADD iterator that changes Groups value and iterates (len(sorted_files) - 1 times)
raw_1 = data_transformation_easy(all_files[0])
# SHOW=True
# fig = raw_1.plot(
#         n_channels=32, 
#         scalings=SCALINGS,
#         show=SHOW
#         )
# info = raw_1.info

# from scipy.io import loadmat
# matfiles = glob.glob(os.path.join(os.getcwd(), 'Depression-Sample-dataset-AIIMS', 'Preeti_PRE_clean.mat'))
# data = {}
# print(matfiles)
# dat = loadmat(matfiles[0])
# raw_2 = np.array(dat['EEG']['data'][0][0])
# raw_2 = mne.io.RawArray(raw_2, info)
# fig = raw_2.plot(
#         n_channels=32, 
#         scalings=2e2,
#         show=SHOW
#         )

raw_1 = g1_preprocess(raw_1)
raw_2 = data_transformation_easy(all_files[1])
raw_2 = g1_preprocess(raw_2)

Creating RawArray with float64 data, n_channels=32, n_times=74999
    Range : 0 ... 74998 =      0.000 ...   149.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hann window with 0.0546 passband ripple and 44 dB stopband attenuation
- Lower transition bandwidth: 0.01 Hz
- Upper transition bandwidth: 11.25 Hz
- Filter length: 155001 samples (310.002 s)



/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:249: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.5s


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Creating RawArray with float64 data, n_channels=32, n_times=74999
    Range : 0 ... 74998 =      0.000 ...   149.996 secs
Ready.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, non-linear phase, causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hann window with 0.0546 passband ripple and 44 dB stopband attenuation
- Lower transition bandwidth: 0.01 Hz
- Upper transition bandwidth: 11.25 Hz
- Filter length: 155001 samples (310.002 s)



/home/vishwani/Downloads/IITD/Depression-IITD/common.ipynb:249: RuntimeWarning: filter_length (155001) is longer than the signal (50001), distortion is likely. Reduce filter length or filter a longer signal.
  ]


EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Excel data

In [7]:
existing_file = f"comparison-results/result-{ACTIVE_SHAM}.xlsx"
df = pd.read_excel(existing_file, header=[0,1])

# main_column_names = ['index', 'swn', 'links', 'asymmetry', 'hubs', 'channels', 'regions']
# subcols = ('theta-g1', 'theta-g2', 'alpha-g1', 'alpha-g2', 'beta-g1', 'beta-g2')

# subcolumn_names = {
#     'index' : ['index'],
#     'swn': subcols,
#     'links': subcols,
#     'asymmetry': subcols,
#     'hubs': ('theta', 'alpha', 'beta'),
#     'channels': ('theta', 'alpha', 'beta'),
#     'regions': ('theta', 'alpha', 'beta')
# }

# columns_tuples = [(main_col, sub_col) for main_col in main_column_names for sub_col in subcolumn_names[main_col]]
# columns = pd.MultiIndex.from_tuples(columns_tuples)
# df = pd.DataFrame(columns=columns)

row_data = {('index', 'index'): f'{ACTIVE_SHAM}-{SAMPLE}-{GROUP1}vs{GROUP2}'}
df

Unnamed: 0_level_0                                          index  \
   Unnamed: 0_level_1 Unnamed: 0_level_1.1                     index   
0                 NaN                  NaN                       NaN   
1                 0.0                  NaN                       NaN   
2                 1.0                  0.0                       NaN   
3                 2.0                  1.0                       NaN   
4                 3.0                  2.0                       NaN   
5                 4.0                  3.0                       NaN   
6                 5.0                  4.0                       NaN   
7                 6.0                  5.0                       NaN   
8                 7.0                  6.0                       NaN   
9                 8.0                  7.0                       NaN   
10                9.0                  8.0                       NaN   
11               10.0                  9.0                       NaN   
12               11.0                 10.0                       NaN   
13               12.0                 11.0                       NaN   
14               13.0                 12.0  Active-Hemlata-PrevsPost   
15               14.0                 13.0    Active-Malti-PrevsPost   
16               15.0                 14.0   Active-Preeti-PrevsPost   
17               16.0                 15.0  Active-Sharifa-PrevsPost   
18               17.0                 16.0   Active-Vinita-PrevsPost   
19               18.0                 17.0      Active-VKS-PrevsPost   
20               19.0                 18.0     Active-Nitu-PrevsPost   
21               20.0                 19.0  Active-Ranjeet-PrevsPost   
22               21.0                 20.0   Active-Resham-PrevsPost   
23               22.0                 21.0   Active-Rithik-PrevsPost   
24               23.0                 22.0    Active-Rohan-PrevsPost   
25               24.0                 23.0    Active-Suman-PrevsPost   
26               25.0                 24.0  Active-Hemlata-PrevsPost   
27               26.0                  NaN    Active-Malti-PrevsPost   

         swn                                                      links  ...  \
    theta-g1  theta-g2  alpha-g1  alpha-g2   beta-g1   beta-g2 theta-g1  ...   
0        NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
1        NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
2        NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
3        NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
4        NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
5        NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
6        NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
7        NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
8        NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
9        NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
10       NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
11       NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
12       NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
13       NaN       NaN       NaN       NaN       NaN       NaN      NaN  ...   
14       NaN       NaN       NaN       NaN       NaN       NaN     27.0  ...   
15  0.328033  0.165424  0.367314  0.286433  0.350444  0.268773    139.0  ...   
16       NaN       NaN  0.250996  0.398823       NaN       NaN     55.0  ...   
17  0.411041  0.318595  0.351765  0.243765  0.413731  0.300443    230.0  ...   
18  0.505946  0.353311  0.519844  0.309701  0.520078  0.258649    225.0  ...   
19  0.280962  0.157745  0.103466  0.193431  0.247594  0.063841     69.0  ...   
20  0.130526  0.382258  0.017914  0.382258  0.151499  0.407903     79.0  ...   
21

### Epoching

In [8]:
duration = 4.0
overlap = 2.0 

samples_per_epoch = int(duration * SFREQ)
samples_per_overlap = int(overlap * SFREQ)

# Manually created events
start, stop = 0, samples_per_epoch
events = []
while stop <= len(raw_1):
    events.append([start, 0, 1]) 
    start += samples_per_overlap
    stop += samples_per_overlap

events = np.array(events)

3.1 Segmenting epochs for Group 1

In [9]:
epochs_1 = mne.Epochs(raw_1, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
# raw_1.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
# epochs_1.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

Not setting metadata
49 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped


In [10]:
# raw_1.plot()
# ica = ICA(n_components=20, method='fastica', random_state=23).fit(raw_1)
# # ica.exclude = [1]
# # raw_clean = ica.apply(raw.copy())
# ica.plot_components()
# ica.plot_properties(raw_1, picks=14)

In [11]:
epochs_2 = mne.Epochs(raw_2, events, tmin=0, tmax=duration, baseline=None, detrend=1,
                    picks=None, preload=True, reject=None)
# raw_2.plot(n_channels=len(raw_1.info['ch_names']), events=events, event_color={1:'r'}, scalings=SCALINGS)
# epochs_2.plot(n_channels=len(raw_1.info['ch_names']), event_color={1:'r'}, events=events, scalings=SCALINGS)

Not setting metadata
49 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 49 events and 2001 original time points ...
5 bad epochs dropped


### 1. PSD graph plot between MDD and Control group

For resting state EEG channel, analyzing the overall average power across channels provides a holistic view of the brain's activity without emphasizing the specificity of individual channels.

In [12]:
# Amplitude plots between both groups
epochs_1_avg = epochs_1['1'].average()
epochs_2_avg = epochs_2['1'].average()
# evokeds = dict(epochs_1=epochs_1)
# fig_MDD = epochs_1_avg.plot(titles=GROUP1, time_unit = 'ms')
# fig_control = epochs_2_avg.plot(titles=GROUP2, time_unit = 'ms')

1.1 PSD using plot_psd

In [13]:
fig, ax = plt.subplots()
#plot_psd uses welch method for continuous data
raw_1.plot_psd(picks=raw_1.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='blue', dB=False)
raw_2.plot_psd(picks=raw_2.info['ch_names'], average=True, fmin=FMIN, fmax=FMAX, ax=ax, show=False, color='red', dB=False)

# dB = True plots PSD in decibels (logarithmic)
# different n_fft compared to psd_array_welch
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power/Frequency (microV^2/Hz)')
ax.set_title('Power Spectral Density Comparison')

ax.text(0.8, 0.9, GROUP1, color='blue', transform=ax.transAxes)
ax.text(0.8, 0.85, GROUP2, color='red', transform=ax.transAxes)

plt.ylim(0, 8)
# plt.show()

fig.savefig(f'comparison-results/PSD - {SAMPLE} - {GROUP1} vs {GROUP2}.png')

NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
Effective window size : 4.096 (s)


### 2. PLI and construction of brain function matrices

In [14]:
# pli method always gives positive correlations
connectivity = {}
def connectivity_matrix(epochs, i):
    return spectral_connectivity_epochs(
    epochs, method='pli', mode='multitaper', sfreq=SFREQ,
    fmin=FREQ_BANDS[mapping[i]][0], fmax=FREQ_BANDS[mapping[i]][1], faverage=True, n_jobs=1)


In [15]:
n_channels = len(raw_1.info['ch_names'])
for i in mapping:
    con = connectivity_matrix(epochs_1, i)
    connectivity[f'{GROUP1}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))

    con = connectivity_matrix(epochs_2, i)
    connectivity[f'{GROUP2}-{mapping[i]}'] = con.get_data().reshape((n_channels, n_channels))


Adding metadata with 3 columns
Connectivity computation...
only using indices for lower-triangular matrix
    computing connectivity for 496 connections
    using t=0.000s..4.000s for estimation (2001 points)
    frequencies: 4.2Hz..8.0Hz (16 points)
    connectivity scores will be averaged for each band
    Using multitaper spectrum estimation with 7 DPSS windows
    the following metrics will be computed: PLI
    computing connectivity for epoch 1
    computing connectivity for epoch 2
    computing connectivity for epoch 3
    computing connectivity for epoch 4
    computing connectivity for epoch 5
    computing connectivity for epoch 6
    computing connectivity for epoch 7
    computing connectivity for epoch 8
    computing connectivity for epoch 9
    computing connectivity for epoch 10
    computing connectivity for epoch 11
    computing connectivity for epoch 12
    computing connectivity for epoch 13
    computing connectivity for epoch 14
    computing connectivity for epo

### 3. Thresholding - M1 -  Small World Index

Preserves small-world properties in both groups to ensure that any observed differences in connectivity are not biased by the thresholding method.
Preserving these properties ensures that information can be transmitted quickly and effectively across the network.

In [16]:
def create_random_graph(n, e):
    adj_matrix = np.zeros((n, n))
    edges = [(i, j) for i in range(n) for j in range(i) if i != j]
    selected_edges = np.random.choice(len(edges), e, replace=False)
    for edge_idx in selected_edges:
        i, j = edges[edge_idx]
        adj_matrix[i, j] = 1
    return adj_matrix

In [17]:
def make_symmetric(arr, n_channels):
    for row in range(n_channels):
        for col in range(n_channels):
            arr[row][col] = arr[col][row]
    return arr

In [18]:
def calculate_swn(mat, threshold):
    tthresh = np.median(mat[mat!=0]) + threshold*np.std(mat[mat!=0])
    print("*****111111******", tthresh)
    binmat= mat > tthresh
    n_connections = np.count_nonzero(binmat == 1)
    print("*****222222******", n_connections)
    binmat = make_symmetric(binmat.astype(int), n_channels)
    g= calculate_avergae_components(nx.from_numpy_array(binmat))
    Lw_binarized, CC_binarized = g[0], g[3]
    return (CC_binarized/Lw_binarized if n_connections else 0), n_connections

In [19]:
WINDOW_SIZE = 5
def moving_average(data, window_size = WINDOW_SIZE):
    cumsum = np.cumsum(data, dtype=float)
    cumsum[window_size:] = cumsum[window_size:] - cumsum[:-window_size]
    return cumsum[window_size - 1:] / window_size

In [20]:
import matplotlib.pyplot as plt
import random

n_perms = 1000
n_network = n_perms//10
thresholds = np.linspace(0, 1.5, 20)
swn_z_1, swn_z_2 = {}, {}
optimal_threshold = {}
optimal_swn_perms, optimal_swn_real1, optimal_swn_real2 = {}, {}, {}
moving_avg1, moving_avg2 = {}, {}

for i in mapping:
    conmat1, conmat2 = connectivity[f'{GROUP1}-{mapping[i]}'], connectivity[f'{GROUP2}-{mapping[i]}']
    swn_z_1[mapping[i]], swn_z_2[mapping[i]] = np.zeros(len(thresholds)), np.zeros(len(thresholds))
    optimal_swn_perms[mapping[i]], optimal_swn_real1[mapping[i]], optimal_swn_real2[mapping[i]] = [[] for _ in range(len(thresholds))], np.zeros(len(thresholds)),  np.zeros(len(thresholds))
    max_diff = 0
    for threshi, threshold in enumerate(thresholds):
        print("00000000", threshi, threshold)
        swn_1, n_connections1 = calculate_swn(conmat1, threshold)
        swn_2, n_connections2 = calculate_swn(conmat2, threshold)

        # Create #n_network random matrices with vertices: n_channels and edges: average of edges of thresholded binary matrix of both groups 
        # And make a list of #n_network lists of clustering coeffs and shortest avg path length
        Crand, Lrand = [], []
        swn_perms = np.zeros(n_perms)
        for perm in range(n_network):
            random_binmat = create_random_graph(n_channels, (n_connections1+n_connections2)//2)
            random_binmat = make_symmetric(random_binmat, n_channels)
            grand = calculate_avergae_components(nx.from_numpy_array(random_binmat))
            Crand.append(grand[3])
            Lrand.append(grand[0])

        print("CHECK", Crand.count(0))
        print("CHECK2", Lrand.count(0))
        # choose any 2 random network from above n_networks #n_perms times 
        for perm in range(n_perms):
            whichnetworks2use = random.sample(range(n_network), 2)
            if Crand[whichnetworks2use[1]] and Lrand[whichnetworks2use[0]]:
                swn_perms[perm] = (Crand[whichnetworks2use[0]] / Crand[whichnetworks2use[1]]) / (Lrand[whichnetworks2use[0]] / Lrand[whichnetworks2use[1]])
            else:
                perm -= 1
    
        swn_rand_avg = np.mean(Crand, axis=0)/np.mean(Lrand, axis=0)
        swn_real_1 = swn_1 / swn_rand_avg # TODO: different random avg or same ?
        swn_real_2 = swn_2 / swn_rand_avg
        val1, val2 = (swn_real_1 - np.mean(swn_perms)) / np.std(swn_perms), (swn_real_2 - np.mean(swn_perms)) / np.std(swn_perms)
        swn_z_1[mapping[i]][threshi] = val1
        swn_z_2[mapping[i]][threshi] = val2


        max_diff = 0
        moving_avg1[mapping[i]] = moving_average(swn_z_1[mapping[i]])
        moving_avg2[mapping[i]] = moving_average(swn_z_2[mapping[i]])
        moving_threshold = moving_average(thresholds)

        # print("$$$$$", optimal_swn_perms[mapping[i]])
        optimal_swn_perms[mapping[i]][threshi] = swn_perms
        optimal_swn_real1[mapping[i]][threshi] =swn_real_1
        optimal_swn_real2[mapping[i]][threshi] =swn_real_2

00000000 0 0.0
*****111111****** 0.6164772727272727
*****222222****** 244
*****111111****** 0.8480113636363638
*****222222****** 248
CHECK 0
CHECK2 0
00000000 1 0.07894736842105263
*****111111****** 0.6377781571307056
*****222222****** 231
*****111111****** 0.8728117507346879
*****222222****** 237
CHECK 0
CHECK2 0
00000000 2 0.15789473684210525
*****111111****** 0.6590790415341384
*****222222****** 217
*****111111****** 0.897612137833012
*****222222****** 221
CHECK 0
CHECK2 0
00000000 3 0.23684210526315788
*****111111****** 0.6803799259375714
*****222222****** 210
*****111111****** 0.9224125249313361
*****222222****** 206
CHECK 0
CHECK2 0
00000000 4 0.3157894736842105
*****111111****** 0.7016808103410042
*****222222****** 202
*****111111****** 0.9472129120296602
*****222222****** 185
CHECK 0
CHECK2 0
00000000 5 0.39473684210526316
*****111111****** 0.7229816947444371
*****222222****** 194
*****111111****** 0.9720132991279843
*****222222****** 155
CHECK 0
CHECK2 0
00000000 6 0.473684210

/tmp/ipykernel_118120/338698987.py:46: RuntimeWarning: divide by zero encountered in scalar divide
  val1, val2 = (swn_real_1 - np.mean(swn_perms)) / np.std(swn_perms), (swn_real_2 - np.mean(swn_perms)) / np.std(swn_perms)
/tmp/ipykernel_118120/338698987.py:46: RuntimeWarning: invalid value encountered in scalar divide
  val1, val2 = (swn_real_1 - np.mean(swn_perms)) / np.std(swn_perms), (swn_real_2 - np.mean(swn_perms)) / np.std(swn_perms)


CHECK 100
CHECK2 100
00000000 19 1.5
*****111111****** 1.0211940763924972
*****222222****** 0
*****111111****** 1.3192187185045219
*****222222****** 0


/tmp/ipykernel_118120/338698987.py:43: RuntimeWarning: invalid value encountered in scalar divide
  swn_rand_avg = np.mean(Crand, axis=0)/np.mean(Lrand, axis=0)


CHECK 100
CHECK2 100
00000000 0 0.0
*****111111****** 0.6363636363636365
*****222222****** 248
*****111111****** 0.8147727272727272
*****222222****** 248
CHECK 0
CHECK2 0
00000000 1 0.07894736842105263
*****111111****** 0.6574089261461982
*****222222****** 243
*****111111****** 0.8388433847372606
*****222222****** 245
CHECK 0
CHECK2 0
00000000 2 0.15789473684210525
*****111111****** 0.6784542159287599
*****222222****** 231
*****111111****** 0.862914042201794
*****222222****** 241
CHECK 0
CHECK2 0
00000000 3 0.23684210526315788
*****111111****** 0.6994995057113216
*****222222****** 216
*****111111****** 0.8869846996663275
*****222222****** 227
CHECK 0
CHECK2 0
00000000 4 0.3157894736842105
*****111111****** 0.7205447954938833
*****222222****** 200
*****111111****** 0.9110553571308609
*****222222****** 214
CHECK 0
CHECK2 0
00000000 5 0.39473684210526316
*****111111****** 0.741590085276445
*****222222****** 175
*****111111****** 0.9351260145953944
*****222222****** 189
CHECK 0
CHECK2 0
00

/tmp/ipykernel_118120/338698987.py:44: RuntimeWarning: invalid value encountered in scalar divide
  swn_real_1 = swn_1 / swn_rand_avg # TODO: different random avg or same ?
/tmp/ipykernel_118120/338698987.py:45: RuntimeWarning: invalid value encountered in scalar divide
  swn_real_2 = swn_2 / swn_rand_avg


CHECK 100
CHECK2 100
00000000 19 1.5
*****111111****** 1.0362241422323089
*****222222****** 0
*****111111****** 1.2721152190988625
*****222222****** 0
CHECK 100
CHECK2 100
00000000 0 0.0
*****111111****** 0.6079545454545454
*****222222****** 248
*****111111****** 0.9004010695187166
*****222222****** 248
CHECK 0
CHECK2 0
00000000 1 0.07894736842105263
*****111111****** 0.6276829516869118
*****222222****** 243
*****111111****** 0.9243615780596902
*****222222****** 233
CHECK 0
CHECK2 0
00000000 2 0.15789473684210525
*****111111****** 0.6474113579192782
*****222222****** 238
*****111111****** 0.9483220866006637
*****222222****** 197
CHECK 0
CHECK2 0
00000000 3 0.23684210526315788
*****111111****** 0.6671397641516446
*****222222****** 223
*****111111****** 0.9722825951416374
*****222222****** 155
CHECK 0
CHECK2 0
00000000 4 0.3157894736842105
*****111111****** 0.6868681703840108
*****222222****** 219
*****111111****** 0.996243103682611
*****222222****** 54
CHECK 0
CHECK2 0
00000000 5 0.3947

In [21]:
# High standardized network metric suggests that our network exhibits properties away from random networks 
for i in mapping:
    plt.plot(thresholds, swn_z_1[mapping[i]], '-o', markerfacecolor='w', label=f'{GROUP1}')
    plt.plot(thresholds, swn_z_2[mapping[i]], '-o', markerfacecolor='w', label=f'{GROUP2}')
    plt.xlabel(f'Threshold (Standard deviations above median) at frequency band {mapping[i]}')
    plt.ylabel(f'SWN_z')
    plt.xlim([0, 1.5])
    plt.legend()
    # plt.show()

### 4. Binarization of functional connectivity

In [23]:
# To maximize the statistical significance of small world networkness for both groups while 
# maximizing the difference between the z-normalized small world networks of the two groups.
f, axarr = plt.subplots(1,3, figsize=(100, 5)) 
for count, i in enumerate(mapping):
    max_diff = 0
    for mov in range(len(moving_threshold)):
        x = np.abs(moving_avg1[mapping[i]][mov] - moving_avg2[mapping[i]][mov])
        if x>max_diff:
            max_diff = x
            optimal_threshold[mapping[i]] = moving_threshold[mov]
            index = mov

    # TODO: save fig file
    p_value_1 = np.mean(optimal_swn_perms[mapping[i]][index+(WINDOW_SIZE//2)] >= optimal_swn_real1[mapping[i]][index+(WINDOW_SIZE//2)])
    p_value_2 = np.mean(optimal_swn_perms[mapping[i]][index+(WINDOW_SIZE//2)] >= optimal_swn_real2[mapping[i]][index+(WINDOW_SIZE//2)])
    print(mapping[i], "p-value: ", p_value_1, p_value_2)


    axarr[count].hist(optimal_swn_perms[mapping[i]][index+(WINDOW_SIZE//2)], bins=50, color='b', alpha=0.7)
    axarr[count].axvline(optimal_swn_real1[mapping[i]][mov+(WINDOW_SIZE//2)], color='m', linewidth=3, label=f'{GROUP1}')
    axarr[count].axvline(optimal_swn_real2[mapping[i]][mov+(WINDOW_SIZE//2)], color='r', linewidth=3, label=f'{GROUP2}')
    axarr[count].set_xlabel(f'{mapping[i]}')
    axarr[count].set_xlim([0,10])
    # axarr[count].set_ylim([0,100])
    axarr[count].legend()
f.suptitle('Random Network distribution for both Pre and Post groups')
# plt.show()

theta p-value:  0.0 1.0
&&&& 25.11675884029522 0.0
alpha p-value:  0.006 1.0
&&&& 5.324683670740924 0.0
beta p-value:  0.0 1.0
&&&& 18.04500596491406 0.0


Text(0.5, 0.98, 'Random Network distribution for both Pre and Post groups')

In [1]:
for i in mapping:
    g1 = calculate_avergae_components(nx.from_numpy_array(optimal_swn_real1[mapping[i]][index+(WINDOW_SIZE//2)]))
    g2 = calculate_avergae_components(nx.from_numpy_array(optimal_swn_real2[mapping[i]][index+(WINDOW_SIZE//2)]))

    if g1[0] and g2[0]:
        row_data[('swn', f'{mapping[i]}-g1')] = g1[3]/g1[0]
        row_data[('swn', f'{mapping[i]}-g2')] = g2[3]/g2[0]
    else:
        row_data[('swn', f'{mapping[i]}-g1')] = None
        row_data[('swn', f'{mapping[i]}-g2')] = None

NameError: name 'mapping' is not defined

In [ ]:
# Find optimal threhsold
binarized_matrix = {}
print(optimal_threshold)
for i in mapping:
    conmat1, conmat2 = connectivity[f'{GROUP1}-{mapping[i]}'], connectivity[f'{GROUP2}-{mapping[i]}']
    tthresh1 = np.median(conmat1[conmat1!=0]) + optimal_threshold[mapping[i]]*np.std(conmat1[conmat1!=0])
    tthresh2 = np.median(conmat2[conmat2!=0]) + optimal_threshold[mapping[i]]*np.std(conmat2[conmat2!=0])
    binarized_matrix[f'{GROUP1}-{mapping[i]}'] = connectivity[f'{GROUP1}-{mapping[i]}'] > tthresh1
    binarized_matrix[f'{GROUP2}-{mapping[i]}'] = connectivity[f'{GROUP2}-{mapping[i]}'] > tthresh2
    print(tthresh1, tthresh2)
print(binarized_matrix.keys())


Find number of links in each graph

In [ ]:
for i in mapping:
    links_1 = 0
    links_2 = 0

    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                links_1+=1
            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                links_2+=1

    print(f'{GROUP1}-{mapping[i]}-number of links: K=', links_1)
    print(f'{GROUP2}-{mapping[i]}-number of links: K=', links_2)

    row_data[('links', f'{mapping[i]}-g1')] = links_1
    row_data[('links', f'{mapping[i]}-g2')] = links_2

### 5. Asymmetry in  each band

In [ ]:
left_region = ['Fp1', 'AF3', 'PO3','O1', 'CP1', 'FC1', 'P3', 'C3', 'F3', 'F7', 'FC5', 'CP5', 'T7', 'P7']
right_region = ['P8', 'T8', 'CP6', 'FC6', 'F8', 'F4', 'C4', 'P4', 'AF4', 'Fp2', 'FC2', 'CP2', 'O2', 'PO4']

for i in mapping:
    lr = 1
    rr = 1
    lrc = 1
    rrc = 1
    for row in range(n_channels):
        for col in range(row+1):
            if binarized_matrix[f'{GROUP1}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lr += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rr += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

            if binarized_matrix[f'{GROUP2}-{mapping[i]}'][row][col] == 1:
                if raw_1.info['ch_names'][row] in left_region or raw_1.info['ch_names'][col] in left_region:
                    lrc += 1
                if raw_1.info['ch_names'][row] in right_region or raw_1.info['ch_names'][col] in right_region:
                    rrc += 1
                # print(raw_1.info['ch_names'][row], raw_1.info['ch_names'][col])

    print(f"{GROUP1}: ",mapping[i], "left region: ", lr, "right region: ", rr, "proportion: ", lr/rr)
    print(f"{GROUP2}: ", mapping[i], "left region: ", lrc, "right region: ", rrc, "proportion: ", lrc/rrc)

    row_data[('asymmetry', f'{mapping[i]}-g1')] = lr/rr
    row_data[('asymmetry', f'{mapping[i]}-g2')] = lrc/rrc

### 6. Difference matrix

In [ ]:
def return_index(channels_list, raw):
    indexes = []
    for channel in channels_list:
        for i in range(len(raw.info['ch_names'])):
            if raw.info['ch_names'][i]==channel:
                indexes.append(i)
    return(indexes)

In [ ]:
central_channels = return_index(['Pz', 'Oz', 'Fz', 'Cz'], raw_1)
central_channels
raw_1.info['bads'] = []

In [ ]:
# f, axarr = plt.subplots(1,3) 
# for count, i in enumerate(mapping):
#     conmat1 = make_symmetric(connectivity[f'{GROUP1}-{mapping[i]}'], n_channels)
#     # conmat2 = make_symmetric(connectivity[f'{GROUP2}-{mapping[i]}'], n_channels)

#     im = axarr[count].imshow(conmat1)
#     axarr[count].set_title(f'Functional connectivity Pre {mapping[i]} in {ACTIVE_SHAM} treatment')
#     axarr[count].set_xticks([_ for _ in range(n_channels)])
#     axarr[count].set_xticklabels(raw_1.info['ch_names'], rotation='vertical')
#     axarr[count].set_yticks([_ for _ in range(n_channels)])
#     axarr[count].set_yticklabels(raw_1.info['ch_names'], rotation='horizontal')
#     axarr[count].grid(True, which='both', linestyle='--', linewidth=0.5)

# f.text(0.5, 0.04, 'X-axis label', ha='center')
# f.text(0.04, 0.5, 'Y-axis label', va='center', rotation='vertical')

# plt.show()

In [ ]:
difference_matrix = {}
for i in mapping:
    # TODO: both difference and excel it in in different column
    difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) 
    # difference_matrix[mapping[i]] = binarized_matrix[f'{GROUP1}-{mapping[i]}'].astype(int) - binarized_matrix[f'{GROUP2}-{mapping[i]}'].astype(int)
    for k in range(n_channels):
        for l in range(n_channels):
            if difference_matrix[mapping[i]][k][l] == -1 or k in central_channels or l in central_channels:
                difference_matrix[mapping[i]][k][l] = 0

    difference_matrix[mapping[i]] = make_symmetric(difference_matrix[mapping[i]], n_channels)

    # # Yellow - Higher synchronisation
    diff_mat = difference_matrix[mapping[i]]
    plt.imshow(diff_mat, interpolation='none')
    plt.title(f'Binary FC difference in {ACTIVE_SHAM} treatment in {mapping[i]}')
    plt.xticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='vertical')
    plt.yticks([_ for _ in range(n_channels)], raw_1.info['ch_names'], rotation='horizontal')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    # plt.show()

    
    # plot_sensors_connectivity(
    #     raw_1.info,
    #     difference_matrix[mapping[i]],
    #     cbar_label=f'{mapping[i]}-Connectivity',
    #     )

In [ ]:
def calculate_hub(mat):
        hub =[]
        for ch, ch_name in enumerate(raw_1.info['ch_names']):
            hub.append(np.count_nonzero(mat[ch] == 1))
        return hub
        # layout = find_layout(raw_1.info)
        # plot_topomap(hub, layout.pos, names=raw_1.info['ch_names'], sphere='eeglab')
        # plt.show()

In [ ]:
edges_threshold = {}
for i in mapping:
    hub = calculate_hub(difference_matrix[mapping[i]])
    edges_threshold[mapping[i]] = np.median(hub) + np.std(hub)
    row_data[('hubs', mapping[i])] = hub
print(edges_threshold)

In [ ]:
selected_channels = {}
for i in mapping:
    selected_channels[mapping[i]] = []
    for ch in range(n_channels):
        if len(nx.from_numpy_array(difference_matrix[mapping[i]]).edges(ch)) > edges_threshold[mapping[i]]:
            selected_channels[mapping[i]].append(raw_1.info['ch_names'][ch])
print(selected_channels)

Create dictionary of Regions in brain vs channels

In [ ]:
regionvschannel = {
    'LC': ['C3', 'CP1', 'CP5', 'FC1', 'FC5'], # Left central 
    'LF': ['FP1', 'AF3', 'F3', 'F7'], # Left frontal
    'LT': ['T7'], # Left temporal
    'LPO': ['PO3',  'P3', 'P7', 'O1'], # Left parietal-occipital
    'RC': ['CP6', 'FC6', 'C4', 'FC2', 'CP2',], # Right central 
    'RF': ['F8', 'F4', 'AF4', 'FP2'], # Right frontal
    'RT': ['T8'], # Right temporal
    'RPO': ['P8', 'P4', 'PO4', 'O2'] , # Right parietal-occipital
}

In [ ]:
channel_to_region = {}
for region, region_channels in regionvschannel.items():
    for channel in region_channels:
        channel_to_region[channel] = region

selected_regions = {}
for band, channels in selected_channels.items():
    regions = set()
    for channel in channels:
        if channel in channel_to_region:
            regions.add(channel_to_region[channel])
    selected_regions[band] = list(regions)

In [ ]:
for i in mapping:
    row_data[('channels', mapping[i])] = selected_channels[mapping[i]]

In [ ]:
for i in mapping:
    row_data[('regions', mapping[i])] = selected_regions[mapping[i]]
row_data

In [18]:
(df)

Empty DataFrame
Columns: [(index, index), (swn, theta-g1), (swn, theta-g2), (swn, alpha-g1), (swn, alpha-g2), (swn, beta-g1), (swn, beta-g2), (links, theta-g1), (links, theta-g2), (links, alpha-g1), (links, alpha-g2), (links, beta-g1), (links, beta-g2), (asymmetry, theta-g1), (asymmetry, theta-g2), (asymmetry, alpha-g1), (asymmetry, alpha-g2), (asymmetry, beta-g1), (asymmetry, beta-g2), (hubs, theta), (hubs, alpha), (hubs, beta), (channels, theta), (channels, alpha), (channels, beta), (regions, theta), (regions, alpha), (regions, beta)]
Index: []

[0 rows x 28 columns]

In [ ]:
# df = df.drop(df.index)
new_row = pd.Series(row_data)
print("length", len(df))
print("new row", new_row)
df.loc[len(df)] = new_row

In [19]:
# # Reset the column index
# df.to_excel(existing_file, index=True, header=True)
# df

Empty DataFrame
Columns: [(index, index), (swn, theta-g1), (swn, theta-g2), (swn, alpha-g1), (swn, alpha-g2), (swn, beta-g1), (swn, beta-g2), (links, theta-g1), (links, theta-g2), (links, alpha-g1), (links, alpha-g2), (links, beta-g1), (links, beta-g2), (asymmetry, theta-g1), (asymmetry, theta-g2), (asymmetry, alpha-g1), (asymmetry, alpha-g2), (asymmetry, beta-g1), (asymmetry, beta-g2), (hubs, theta), (hubs, alpha), (hubs, beta), (channels, theta), (channels, alpha), (channels, beta), (regions, theta), (regions, alpha), (regions, beta)]
Index: []

[0 rows x 28 columns]